In [1]:
import pandas as pd
df_ground_truth = pd.read_csv('ground-truth-data.csv')

In [2]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [3]:
ground_truth[73]

{'question': 'Assuming I am selected for financial assistance from the "NITW Endowment Fund-1997 Batch", what are the permitted uses of the funds? Can the funds be used for tuition fees, hostel expenses, study materials, project expenses, or a combination thereof? Are there any restrictions on how the funds can be utilized?',
 'url': 'https://nitw.ac.in/api/static/files/Notice_of_NITW_Endowment_Fund-1997_Batch_2024-11-12-11-0-47.pdf'}

In [4]:
import json
with open ('database.json' , 'rt') as f_in:
    documents = json.load(f_in)

In [5]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "url": {"type": "text"},
            "date": {"type": "text"},
            "title": {"type": "text"},
            "info": {"type": "text"}
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)


ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7fc741232140>: Failed to establish a new connection: [Errno 111] Connection refused))

In [24]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/89 [00:00<?, ?it/s]

In [36]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["title", "info"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [6]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [8]:
from tqdm.auto import tqdm

In [26]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['url']
    results = elastic_search(query=q['question'])
    relevance = [d['url'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/435 [00:00<?, ?it/s]

In [10]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['url']
        results = search_function(q)
        relevance = [d['url'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [37]:
evaluate(ground_truth, lambda q: elastic_search(q['question']))

  0%|          | 0/435 [00:00<?, ?it/s]

{'hit_rate': 0.7011494252873564, 'mrr': 0.8283908045977005}

In [22]:
evaluate(ground_truth, lambda q: search(q['question']))

  0%|          | 0/435 [00:00<?, ?it/s]

{'hit_rate': 0.593103448275862, 'mrr': 0.4709961685823757}

In [11]:
import minsearch

In [23]:
Index = minsearch.Index(
    text_fields = [ "title" ,  "info" ],
    keyword_fields= []
)
Index.fit(documents)
def search (query):
        boost = {"title": 1 , 'info': 1}
        results = Index.search(
            query=query,
            boost_dict= boost,
            num_results=5
        )

        return results

In [ ]:
from sentence_transformers import SentenceTransformer